<a href="https://colab.research.google.com/github/ganeshchakravarthy/projects/blob/main/Fine_tune_Alexa_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets torch scikit-learn pandas

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving simple_nlp_dataset.csv to simple_nlp_dataset.csv


In [ ]:
import pandas as pd
import torch
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer
from datasets import Dataset
from sklearn.model_selection import train_test_split

In [ ]:
#Load
df = pd.read_csv("simple_nlp_dataset.csv")

In [ ]:
# Dataset to hugging face format
dataset = Dataset.from_pandas(df)

In [ ]:
model_name = "bert-base-uncased"
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=5)  # Assuming 5 categories
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
train_size = int(0.8 * len(tokenized_datasets))
train_data = tokenized_datasets.select(range(train_size))
val_data = tokenized_datasets.select(range(train_size, len(tokenized_datasets)))

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
training_args = TrainingArguments(
    output_dir="./bert_results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=4,  # Adjust based on Colab RAM
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    logging_dir="./logs",
    save_total_limit=2,
    logging_steps=1,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data
)

trainer.train()  # Start Training


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss
1,1.548600,1.491889
2,1.409200,1.471232
3,1.517800,1.452917


TrainOutput(global_step=6, training_loss=1.4742056330045064, metrics={'train_runtime': 233.6131, 'train_samples_per_second': 0.103, 'train_steps_per_second': 0.026, 'total_flos': 6314835419136.0, 'train_loss': 1.4742056330045064, 'epoch': 3.0})

In [ ]:

eval_results = trainer.evaluate()
print("\nEvaluation Results:", eval_results)


trainer.save_model("./fine_tuned_bert")
print("Model saved successfully!")


📊 Evaluation Results: {'eval_loss': 1.4529166221618652, 'eval_runtime': 3.6635, 'eval_samples_per_second': 0.546, 'eval_steps_per_second': 0.273, 'epoch': 3.0}
✅ Model saved successfully!


In [ ]:
import os
model_path = "./fine_tuned_bert"
print("Saved Model Files:", os.listdir(model_path))

Saved Model Files: ['training_args.bin', 'model.safetensors', 'config.json']


In [ ]:
from transformers import AutoTokenizer

model_path = "./fine_tuned_bert"

# Load the original tokenizer again
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Save the tokenizer inside the same model directory
tokenizer.save_pretrained(model_path)

print("Tokenizer saved successfully!")

✅ Tokenizer saved successfully!


In [ ]:
from transformers import BertForSequenceClassification, AutoTokenizer

model_path = "./fine_tuned_bert"
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

print("Model & Tokenizer Loaded Successfully!")

✅ Model & Tokenizer Loaded Successfully!


In [ ]:
import torch

# Function to test the model
def predict_text(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits).item()

    return predicted_class

# Test with new queries
test_sentences = [
    "Alexa, play some relaxing music.",
    "What’s the weather like in New York?",
    "Set an alarm for 6 AM.",
    "Tell me a joke.",
    "Translate 'good morning' to French."
]

# Get predictions
for sentence in test_sentences:
    print(f"📝 Input: {sentence}")
    print(f"🔍 Predicted Class: {predict_text(sentence)}\n")

📝 Input: Alexa, play some relaxing music.
🔍 Predicted Class: 0

📝 Input: What’s the weather like in New York?
🔍 Predicted Class: 1

📝 Input: Set an alarm for 6 AM.
🔍 Predicted Class: 3

📝 Input: Tell me a joke.
🔍 Predicted Class: 3

📝 Input: Translate 'good morning' to French.
🔍 Predicted Class: 4



In [ ]:
import torch.nn.functional as F

def predict_with_confidence(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    probabilities = F.softmax(outputs.logits, dim=1)  # Convert to probabilities
    predicted_class = torch.argmax(probabilities).item()

    return predicted_class, probabilities.numpy()

# Test with a sample
sentence = "Alexa, play Django Unchained."
predicted_class, confidence = predict_with_confidence(sentence)

print(f"📝 Input: {sentence}")
print(f"🔍 Predicted Class: {predicted_class}")
print(f"📊 Confidence Scores: {confidence}")

📝 Input: Alexa, play Django Unchained.
🔍 Predicted Class: 1
📊 Confidence Scores: [[0.20360258 0.22344984 0.18845664 0.21091601 0.17357488]]
